# How the PWP3D/RBOT approximations work

See the GitHub branch for further explanation and what I believe are sign errors (that balance each other out) in the PWP3D and RBOT papers.

In short: I believe that the true value of $\frac{\partial \Phi}{\partial \xi}$ should be the negation of what the authors write, and that its missing negative sign would balance out the other one.
The physical/intuitive interpretation of how the algorithm works, at least for BG pixels, is as follows. We will just work with BG pixels for now to simplify some things (e.g., so that "signed distance" from the contour can be assumed to be positive, making it just "distance") and because the interpretation seems a bit more "sensible" for BG pixels; for FG pixels, because they use the 3D point "under" the current pixel rather than the 3D point "at" the nearest contour pixel for calculations, there are some parts that I cannot fully incorporate into this interpretation, nor do I have a better alternative one yet.

The RBOT paper represents $\mathbf{x}$ as a function $\mathbf{x}(\xi)$ of $\xi$, so that $\partial\mathbf{x} / \partial \xi$ is not 0 in general, and so that:

$$
\frac{\partial\Phi}{\partial \xi} = \frac{\partial \Phi}{\partial \mathbf{x}} \frac{\partial \mathbf{x}}{\partial \xi}
$$

However, I find it more intuitive to view $\mathbf{x}$ as constant, so that, not only do they not change colour (as in the previous, flawed interpretation) but they also do not change position in 2D/3D space either. Then we'd have $\partial \mathbf{x} / \partial \xi = 0$, and so that $\partial \Phi / \partial \xi \neq 0$, we'll instead represent $\Phi$ as a function of both $\mathbf{x}$ and $\xi$, i.e., $\Phi(\mathbf{x}, \xi)$. We'll also introduce a new function, $n(\mathbf{x}, \xi)$, which outputs the 2D coordinates of the nearest pixel to $\mathbf{x}$ on the silhouette contour for the pose $\xi$.
Lastly, we'll use angle brackets to denote normalization; that is, for a vector $\mathbf{v}$, we'll denote $\langle \mathbf{v} \rangle = \mathbf{v} / \lVert \mathbf{v} \rVert$.

Then, we can say that:
$$
\begin{aligned}
\Phi(\mathbf{x}, \xi)
    &= \lVert n(\mathbf{x}, \xi) - \mathbf{x} \rVert \\
    &= \sqrt{ (n_x(\mathbf{x}, \xi) - x)^2 + (n_y(\mathbf{x}, \xi) - y)^2} \\
\frac{\partial \Phi(\mathbf{x}, \xi)}{\partial \xi}
    &= \frac{\frac{\partial}{\partial \xi} \left((n_x(\mathbf{x}, \xi) - x)^2 + (n_y(\mathbf{x}, \xi) - y)^2\right)}{2\lVert n(\mathbf{x}, \xi) - \mathbf{x} \rVert}\\
    &= \frac{2((n_x(\mathbf{x}, \xi) - x)\frac{\partial n_x}{\partial \xi} + (n_y(\mathbf{x}, \xi) - y)\frac{\partial n_y}{\partial \xi} )}{2\lVert n(\mathbf{x}, \xi) - \mathbf{x} \rVert}\\
    &= \frac{(n(\mathbf{x}, \xi) - \mathbf{x})^T \, \frac{\partial n(\mathbf{x}, \xi)}{\partial \xi}}{\lVert n(\mathbf{x}, \xi) - \mathbf{x} \rVert}\\
    &= \langle \,n(\mathbf{x}, \xi) - \mathbf{x}\,\rangle^T\, \frac{\partial n(\mathbf{x}, \xi)}{\partial \xi}
\end{aligned}
$$

Assuming that a unique $n(\mathbf{x}, \xi)$ exists for $\mathbf{x}$, then $\frac{\partial \Phi}{\partial \mathbf{x}} = \nabla \Phi = \langle \,\mathbf{x} - n(\mathbf{x}, \xi) \,\rangle = -\langle \,n(\mathbf{x}, \xi) - \mathbf{x}\,\rangle$. Explanation/justification can be found [here](https://math.stackexchange.com/questions/2358675/gradient-of-the-distance-of-point-and-set) and [here](https://math.stackexchange.com/questions/693520/the-gradient-of-a-distance-function). Thus, the finite-difference-calculated $\frac{\partial \Phi(\mathbf{x})}{\partial \mathbf{x}}$ calculated in the RBOT paper is essentially the same as $-\langle \,n(\mathbf{x}, \xi) - \mathbf{x}\,\rangle$, which takes care of balancing out the other missing sign. And if you read through the RBOT code (in the `operator()` method of the `Parallel_For_computeJacobiansGN` class in `optimization_engine.h`), then you can see that what the RBOT paper reports as $\frac{\partial \mathbf{x}}{\partial \xi}$ (which they represent by the equivalent $\frac{\partial \pi}{\partial \xi}$, with $\pi$ being the projection function taking 3D points to 2D) is _actually_ just $\frac{\partial n}{\partial \xi}$ for BG pixels _so long as_ the following assumption is made.

The assumption is that WHICH 3D point on the model's surface becomes the nearest contour 2D point (under projection) to the (static) pixel $\mathbf{x}$ won't change, but that WHERE said 3D point is will change.

Viewed another way: it's like each pixel gets its own 1-point, radial SDT that it cares about. Or: it's like each pixel gets its own 1-vertex mesh that it is using to approximate the SDT derivatives.

Now, the interpretation for FG pixels is a bit less clear, since they assume the nearest point would move by the same amount as the 3D point hit by a ray through the current pixel, and I'm not sure when that would be more accurate than just using the same model as the BG... but nonetheless, even just having the intuition for the BG might help.

I _think_ my above interpretation/intuition is fairly decent, but maybe there's a better one; the authors never (as far as I can tell) clearly state that the above is being done, and further, the seemingly-mismatched sign stuff obscures such an interpretation.

I decided to, as a test, see what happens in the RBOT code if you use the closest contour xyz when looking at the FG in the same way they are used for getting 3D points for the BG. On some objects, like the cube, koala candy, etc., accuracy is increased, while for others, like the baking soda and clown, accuracy is decreased. In both cases, though, the difference was fairly small. I still need to test more before making statements about a pattern, but so far, it's more obviously the sharp-edged prism-like objects for whom accuracy increased.

<hr>

Anyway, to recap:

$$
\frac{\partial \Phi(\mathbf{x}, \xi)}{\partial \xi} = \langle \,n(\mathbf{x}, \xi) - \mathbf{x}\,\rangle^T\, \frac{\partial n(\mathbf{x}, \xi)}{\partial \xi}
$$

with the first vector in the dot product being already taken care of by the RBOT code. Now, what about the other vector, the $\frac{\partial n (\mathbf{x}, \xi)}{\partial \xi}$? Is there any way to take advantage of the single-point-model realization or to improve upon its accuracy?
(Right now, it's like each BG pixel gets its own 1-point SDT that it cares about.)

I don't think this realization about how PWP3D/RBOT treat things really opens up any GPU optimizations/precomputations for their approach that I was not already in the process
of implementing via the mesh-based SDT approach; _maybe_ the fact that only the *unit* dir matters would let
me make a few more values constant rather than interpolated, but that's about it.

Treating the silhouette as a set of infinitesimal points separated in space would not change anything, because
the derivative of the piecewise min() function would still just look at only the closest point in isolation.

Maybe a polyline would be slightly more accurate, or estimating local 2D curvature at each iteration, but any image processing becomes really expensive, especially on mobile. So it'd be better to precompute these values and store them in the mesh... at which point, it seems better to modify the sparse approach of SRT3D and the like instead of trying to find a better dense approach.



## Wait, is an improvement even possible?

Because a few test examples that I _thought_ would yield different results for the single-point approximation compared to the "ground truth" did not in fact do so. So let's see if we can find one! If not... guess I need to pick something else to focus on more for the thesis. 🙁

Let's first start with the distance from a pixel to an infinitely-long straight line, and let's only worry about 2D motion for now. We'll parameterize the line by a point $(x_0, y_0)$ and a direction $\theta \in [0, 2\pi)$. We could, instead of $\theta$, use a 2nd point (four variables), but that makes setting a "fixed" point on the line harder, so sticking with three variables seems better.

The constant $(x_p, y_p)$ represents our pixel.


In [2]:
import numpy as np               # For fast floating-point array computations
import time                      # For getting CPU time & whatnot
import random
import matplotlib.pyplot as plt  # For graphs
import math                      # For non-symbolic math (like float sqrt)


# Sympy is the symbolic math library.
import sympy
from sympy import simplify, solve, Eq # Sympy functions we'll use especially often.

# For displaying math nicely
from IPython.display import display
sympy.init_printing()

x0, xp, y0, yp = sympy.symbols('x0, x_p, y0, y_p', real=True)
lambda0, lambda1 = sympy.symbols('lambda0, lambda1', real=True)


p0 = sympy.Matrix([x0, y0])
p1 = sympy.Matrix([xp, yp])

s0, s1 = sympy.symbols('s0, s1', real=True, positive=True)

theta = sympy.Symbol('theta', real=True)


In [4]:
HR_STR = "---------------------------------------------------------------------"
ray_dir = sympy.Matrix([sympy.cos(theta), sympy.sin(theta)])
const_subs = {x0: s0, y0: s1, theta: lambda0}
projection_len = (p1 - p0).dot(ray_dir)
const_proj_len = projection_len.subs(const_subs)


nearest = p0 + projection_len * ray_dir
const_nearest = p0 + const_proj_len * ray_dir

diff = (nearest - p1)
dist = diff.dot(diff)

const_diff = (const_nearest - p1)
const_dist = const_diff.dot(const_diff)

display(simplify(nearest))
display(simplify(dist))

grad = sympy.Matrix([sympy.diff(dist, var) for var in [x0, y0, theta]])
display(simplify(grad))


const_grad = sympy.Matrix([sympy.diff(const_dist, var) for var in [x0, y0, theta]])
display(simplify(const_grad))

print(HR_STR)
grad_for_subs = grad.subs(const_subs)
const_grad_for_subs = const_grad.subs(const_subs)
display(simplify(grad_for_subs))
display(simplify(const_grad_for_subs))

⎡x₀ - ((x₀ - xₚ)⋅cos(θ) + (y₀ - yₚ)⋅sin(θ))⋅cos(θ)⎤
⎢                                                 ⎥
⎣y₀ - ((x₀ - xₚ)⋅cos(θ) + (y₀ - yₚ)⋅sin(θ))⋅sin(θ)⎦

                                                         2                    
(-x₀ + xₚ + ((x₀ - xₚ)⋅cos(θ) + (y₀ - yₚ)⋅sin(θ))⋅cos(θ))  + (-y₀ + yₚ + ((x₀ 

                                        2
- xₚ)⋅cos(θ) + (y₀ - yₚ)⋅sin(θ))⋅sin(θ)) 

⎡                                                     -x₀⋅cos(2⋅θ) + x₀ + xₚ⋅c
⎢                                                                             
⎢                                                     -x₀⋅sin(2⋅θ) + xₚ⋅sin(2⋅
⎢                                                                             
⎢  2                                                                       2  
⎣x₀ ⋅sin(2⋅θ) - 2⋅x₀⋅xₚ⋅sin(2⋅θ) - 2⋅x₀⋅y₀⋅cos(2⋅θ) + 2⋅x₀⋅yₚ⋅cos(2⋅θ) + xₚ ⋅s

os(2⋅θ) - xₚ - y₀⋅sin(2⋅θ) + yₚ⋅sin(2⋅θ)                                      
                                                                              
θ) + y₀⋅cos(2⋅θ) + y₀ - yₚ⋅cos(2⋅θ) - yₚ                                      
                                                                              
                                                  2                           
in(2⋅θ) + 2⋅xₚ⋅y₀⋅cos(2⋅θ) - 2⋅xₚ⋅yₚ⋅cos(2⋅θ) - y₀ ⋅sin(2⋅θ) + 2⋅y₀⋅yₚ⋅sin(2⋅θ

                ⎤
                ⎥
              

⎡             2⋅x₀ - 2⋅xₚ - 2⋅((s₀ - xₚ)⋅cos(λ₀) + (s₁ - yₚ)⋅sin(λ₀))⋅cos(θ)  
⎢                                                                             
⎢             2⋅y₀ - 2⋅yₚ - 2⋅((s₀ - xₚ)⋅cos(λ₀) + (s₁ - yₚ)⋅sin(λ₀))⋅sin(θ)  
⎢                                                                             
⎣2⋅((s₀ - xₚ)⋅cos(λ₀) + (s₁ - yₚ)⋅sin(λ₀))⋅(x₀⋅sin(θ) - xₚ⋅sin(θ) - y₀⋅cos(θ) 

            ⎤
            ⎥
            ⎥
            ⎥
+ yₚ⋅cos(θ))⎦

---------------------------------------------------------------------


⎡                                                        -s₀⋅cos(2⋅λ₀) + s₀ - 
⎢                                                                             
⎢                                                        -s₀⋅sin(2⋅λ₀) + s₁⋅co
⎢                                                                             
⎢  2                                                                          
⎣s₀ ⋅sin(2⋅λ₀) - 2⋅s₀⋅s₁⋅cos(2⋅λ₀) - 2⋅s₀⋅xₚ⋅sin(2⋅λ₀) + 2⋅s₀⋅yₚ⋅cos(2⋅λ₀) - s

s₁⋅sin(2⋅λ₀) + xₚ⋅cos(2⋅λ₀) - xₚ + yₚ⋅sin(2⋅λ₀)                               
                                                                              
s(2⋅λ₀) + s₁ + xₚ⋅sin(2⋅λ₀) - yₚ⋅cos(2⋅λ₀) - yₚ                               
                                                                              
 2                                                       2                    
₁ ⋅sin(2⋅λ₀) + 2⋅s₁⋅xₚ⋅cos(2⋅λ₀) + 2⋅s₁⋅yₚ⋅sin(2⋅λ₀) + xₚ ⋅sin(2⋅λ₀) - 2⋅xₚ⋅yₚ

                          ⎤
                      

⎡               2⋅s₀ - 2⋅xₚ - 2⋅((s₀ - xₚ)⋅cos(λ₀) + (s₁ - yₚ)⋅sin(λ₀))⋅cos(λ₀
⎢                                                                             
⎢               2⋅s₁ - 2⋅yₚ - 2⋅((s₀ - xₚ)⋅cos(λ₀) + (s₁ - yₚ)⋅sin(λ₀))⋅sin(λ₀
⎢                                                                             
⎣2⋅((s₀ - xₚ)⋅cos(λ₀) + (s₁ - yₚ)⋅sin(λ₀))⋅(s₀⋅sin(λ₀) - s₁⋅cos(λ₀) - xₚ⋅sin(λ

)               ⎤
                ⎥
)               ⎥
                ⎥
₀) + yₚ⋅cos(λ₀))⎦

In [11]:
sample_sit = {s0: 1, s1: 10, lambda0: -sympy.pi/6, xp: 0, yp: -1}

display(simplify(grad_for_subs.subs(sample_sit)))
display(simplify(const_grad_for_subs.subs(sample_sit)))



⎡ 1   11⋅√3 ⎤
⎢ ─ + ───── ⎥
⎢ 2     2   ⎥
⎢           ⎥
⎢  √3   33  ⎥
⎢  ── + ──  ⎥
⎢  2    2   ⎥
⎢           ⎥
⎣-11 + 60⋅√3⎦

⎡ 1   11⋅√3 ⎤
⎢ ─ + ───── ⎥
⎢ 2     2   ⎥
⎢           ⎥
⎢  √3   33  ⎥
⎢  ── + ──  ⎥
⎢  2    2   ⎥
⎢           ⎥
⎣-11 + 60⋅√3⎦

In [12]:
display(simplify(grad_for_subs - const_grad_for_subs))

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

Well, damn it. It looks like 2D translation/rotation for a line is already perfectly simulated at the point of interest by just looking at the nearest point.

I guess next I'll test a circle, then I'll test 3D for these simple shapes, and then... idk, I guess search for a general proof, since if it's true, then I doubt I'm the first to prove it. But how would I google it?


## Overestimating/Underestimating


If the single-point approximation isn't already perfect, then I need to confirm this next thought, but I think the PWP3D/RBOT version never really _underestimates_ the distance derivative magnitude; it can only ever _overestimate_, because the existence of other points will never _increase_ the true distance. Not sure if there's any value to this realization, though...

# Principal Curvatures: cylinder case

So, let's see if we can improve the SRT3D approach a bit by considering principal curvatures.

We will start by assuming the cylindrical case, for two reasons:

First, the math is a lot simpler, so it'll give a better idea of feasibility before we go too far.

Second, as the various "large displacement" papers showed, it's mostly the "out-of-frame" rotations that SRT3D sometimes struggles with. So even though curvature would still improve the realism of the "2D" derivative approximations in some cases, perhaps only considering the "out of frame" curvature would be a good compromise to consider.


## The math for the cylinder case

### Projection and Further Approximation
To save on some typesetting, I'll use "p" instead of $\xi$ below.

The unit dir to the nearest point is already accurate/efficient from the old methods (both sparse and dense), so we just need to worry about "dn/dp".

Now... there's a slight complication in that the closest point in 2D is not necessarily the same as the closest point in 3D.

Consider, for example, a camera ray through the image centre and a cylinder that's aligned with the camera but shifted to the right. Even though infinitely many cylinder points are the same shortest 3D distance from the ray, in 2D, the points further away from the camera will seem closer to the centre pixel.

This presents multiple problems... if we assume infinite cylinders, then there are cases like this where the shortest length is undefined (or, I guess, zero, if you consider limits). Finite cylinders add extra complexity, even if just for finding out what height of one should be.

So, should we just find the closest 3D point, and then project, as an approximation? Or should we look directly at what optimizes the perspective projection distance?

Let's start with the first, because it seems a bit simpler. And hey, maybe minimizing "nearest 3D distance" could end up being better in the long run anyway, by certain metrics? I mean, if one part of an object is much further away from the camera than another, then it will have way



### The projection-delaying approximation:

Let's call "z" the nearest 3D point (to a line going from the camera through the pixel), let L(x,y) be this line, and use "P" for projection (just to reduce tex-ing at the moment).

Then:
```
n(x,y,p) = P(z(L(x,y), p))

dn/dp = dP/dz dz/dp
```

The dP/dz is already derived in the other papers. So we just have a... "fancier" dz/dp to find.

Let's first just consider a stationary cylinder with radius "r" and unit dir "v". let's give the line a point "q" and unit dir "u".

What we're really looking for, then, is the closest point on
[finish this later]

### The exact projection-based calculation:

Assuming an infinite cylinder, what does the contour look like after perspective projection? Obviously, just two straight lines. And in our case, we only care about one of those lines.

So, finding the closest point on a 2D line to a point is quite straight-forward. The question is how to get this line from a cylinder and pose, and what the derivatives are after.

If the cylinder were z-axis-aligned, then its parametric form is:
`(x - a)^2 + (y - b)^2 - r^2 = 0`.

If we let I2 be a 2x2 identity matrix , and we let "x" be the vector `[x, y]`, and finally we let alpha be the vector `[a, b]`, then this is just `x^T A x - 2*dot(alpha, x) + a^2 + b^2 - r^2 = 0`.

If we consider points along a ray `(e + t*u)`, but just the xy component so that it is 2D, then this is



(Rx + alpha)^T P (Rx + alpha) - r^2 = 0

x^t (R^t P R) x + 2 alpha^T P R x + a^2 + b^2 - r^2


In [ ]:
https://www.cambridge.org/core/services/aop-cambridge-core/content/view/B131C04079690F5EB1546800889A03DE/S1757748900001961a.pdf/div-class-title-principal-radii-of-curvature-at-a-point-on-an-ellipsoid-div.pdf

https://www.eecg.toronto.edu/~pagiamt/kcsmith/00169661.pdf

(("lines of curvature" OR "curvature lines") AND ("mesh" OR "tris" OR "triangular" OR "polygonal"))



SyntaxError: invalid decimal literal (<ipython-input-6-31bc3c63f802>, line 3)